In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
import random

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce

from torchvision.transforms import Resize
from syn_DI_dataset import make_dataset, make_dataloader


## Load ALL Models

### RGB

In [2]:
class rgb_feature_extractor(nn.Module):
    def __init__(self, rgb_model):
        super(rgb_feature_extractor, self).__init__()
        self.part = nn.Sequential(*list(rgb_model.children())[:-2])
    def forward(self, x):
        x = self.part(x).view(x.size(0), 512, -1)
        x = x.permute(0, 2, 1)
        return x

In [3]:
from RGB_benchmark.rgb_ResNet18.RGB_ResNet import *

rgb_model = RGB_ResNet18()
rgb_model.load_state_dict(torch.load('./RGB_benchmark/rgb_ResNet18/RGB_Resnet18_copy.pt'))
rgb_extractor = rgb_feature_extractor(rgb_model)
rgb_extractor.eval()

rgb_feature_extractor(
  (part): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
      (3): ReLU()
      (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
      (5): ReLU()
      (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
    )
    (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (5): Sequential(
      (0): Block(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (batch_norm2): B

In [25]:
test = torch.rand(1, 3, 480, 640)
out = rgb_extractor(test)
print(out.shape)

torch.Size([1, 49, 512])


### Depth

In [4]:
class depth_feature_extractor(nn.Module):
    def __init__(self, depth_model):
        super(depth_feature_extractor, self).__init__()
        self.part = nn.Sequential(*list(depth_model.children())[:-2])
    def forward(self, x):
        x = self.part(x).view(x.size(0), 512, -1)
        x = x.permute(0, 2, 1)
        return x

In [5]:
from depth_benchmark.depth_ResNet18 import *
depth_model = Depth_ResNet18()
depth_model.load_state_dict(torch.load('depth_benchmark/depth_Resnet18.pt'))
depth_extractor = depth_feature_extractor(depth_model)
depth_extractor.eval()

depth_feature_extractor(
  (part): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
      (3): ReLU()
      (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
      (5): ReLU()
      (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
    )
    (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (5): Sequential(
      (0): Block(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (batch_norm2):

In [28]:
test = torch.rand(1, 3, 480, 640)
out = depth_extractor(test)
print(out.shape)

torch.Size([1, 49, 512])


### mmWave

In [6]:
class mmwave_feature_extractor(nn.Module):
    def __init__(self, mmwave_model):
        super(mmwave_feature_extractor, self).__init__()
        self.part = nn.Sequential(*list(mmwave_model.children())[:-1])
    def forward(self, x):
        x, _ = self.part(x)
        return x

In [7]:
from mmwave_benchmark.mmwave_point_transformer import *
mmwave_model = mmwave_PointTransformerReg()
mmwave_model.load_state_dict(torch.load('mmwave_benchmark/mmwave_all_random.pt'))
mmwave_extractor = mmwave_feature_extractor(mmwave_model)
mmwave_extractor.eval()

mmwave_feature_extractor(
  (part): Sequential(
    (0): Backbone(
      (fc1): Sequential(
        (0): Linear(in_features=5, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
      )
      (transformer1): TransformerBlock(
        (fc1): Linear(in_features=32, out_features=128, bias=True)
        (fc2): Linear(in_features=128, out_features=32, bias=True)
        (fc_delta): Sequential(
          (0): Linear(in_features=3, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
        )
        (fc_gamma): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
        )
        (w_qs): Linear(in_features=128, out_features=128, bias=False)
        (w_ks): Linear(in_features=128, out_features=128, bias=False)
        (w_vs): Linear(in_features=128, o

In [31]:
test = torch.rand(1, 57, 5)
out = mmwave_extractor(test)
print(out.shape)

torch.Size([1, 57, 512])


### Lidar

In [8]:
class lidar_feature_extractor(nn.Module):
    def __init__(self, lidar_model):
        super(lidar_feature_extractor, self).__init__()
        # self.model = lidar_model
        npoints, nblocks, nneighbor, n_c, d_points = 1024, 5, 16, 51, 3
        self.fc1 = lidar_model.backbone.fc1
        self.transformer1 = lidar_model.backbone.transformer1
        self.transition_downs = nn.ModuleList()
        self.transformers = nn.ModuleList()
        for i in range(nblocks - 4):
            channel = 32 * 2 ** (i + 1)
            self.transition_downs.append(lidar_model.backbone.transition_downs[i])
            self.transformers.append(lidar_model.backbone.transformers[i])
        self.nblocks = nblocks
    
    def forward(self, x):
        xyz = x[..., :3]
        points = self.transformer1(xyz, self.fc1(x))[0]

        xyz_and_feats = [(xyz, points)]
        for i in range(self.nblocks - 4):
            xyz, points = self.transition_downs[i](xyz, points)
            points = self.transformers[i](xyz, points)[0]
            xyz_and_feats.append((xyz, points))
        points = points.view(points.size(0), -1, 512)
        return points

In [9]:
from lidar_benchmark.lidar_point_transformer import *
lidar_model = PointTransformerReg()
lidar_model.load_state_dict(torch.load('lidar_benchmark/lidar_all_random.pt'))
lidar_extractor = lidar_feature_extractor(lidar_model)
lidar_extractor.eval()

lidar_feature_extractor(
  (fc1): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (transformer1): TransformerBlock(
    (fc1): Linear(in_features=32, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=32, bias=True)
    (fc_delta): Sequential(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
    )
    (fc_gamma): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
    )
    (w_qs): Linear(in_features=512, out_features=512, bias=False)
    (w_ks): Linear(in_features=512, out_features=512, bias=False)
    (w_vs): Linear(in_features=512, out_features=512, bias=False)
  )
  (transition_downs): ModuleList(
    (0): TransitionDown(
      (sa): PointNetSetAbstraction

In [34]:
test = torch.rand(1, 365, 3)
out = lidar_extractor(test)
print(out.shape)

torch.Size([1, 32, 512])


### WiFI-CSI

In [10]:
class csi_feature_extractor(nn.Module):
    def __init__(self, model):
        super(csi_feature_extractor, self).__init__()
        self.part = nn.Sequential(
            model.encoder_conv1,
            model.encoder_bn1,
            model.encoder_relu,
            model.encoder_layer1,
            model.encoder_layer2,
            model.encoder_layer3,
            model.encoder_layer4, 
            # torch.nn.AvgPool2d((1, 4))
        )
    def forward(self, x):
        x = x.unsqueeze(1)
        x = torch.transpose(x, 2, 3) #16,2,114,3,32
        x = torch.flatten(x, 3, 4)# 16,2,114,96
        torch_resize = Resize([136,32])
        x = torch_resize(x)
        x = self.part(x).view(x.size(0), 512, -1)
        x = x.permute(0, 2, 1)
        return x

In [11]:
import sys
sys.path.insert(0, './CSI_benchmark')
csi_model = torch.load('CSI_benchmark/protocol3_random_1.pkl')
csi_extractor = csi_feature_extractor(csi_model)
csi_extractor.eval()

csi_feature_extractor(
  (part): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kerne

In [37]:
test = torch.rand(32, 3, 114, 10).cuda()
out = csi_extractor(test) 
print(out.shape)

c:\Users\Chen_Xinyan\anaconda3\envs\pytorch\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([32, 68, 512])


# main

In [12]:
rgb_extractor = rgb_extractor.cuda()
# input shape: B, 3, 480, 640; output shape: B, 49, 512
depth_extractor = depth_extractor.cuda()
# input shape: B, 3, 480, 640; output shape: B, 49, 512
mmwave_extractor = mmwave_extractor.cuda()
# input shape: B, n, 5; output shape: 1, n, 512
lidar_extractor = lidar_extractor.cuda()
# input shape: B, n, 3; output shape: B, 32, 512
csi_extractor = csi_extractor.cuda()
# input shape: B, 3, 114, 10; output shape: B, 68, 512

In [13]:
class feature_extrator(nn.Module):
    def __init__(self, rgb_extractor, depth_extractor, mmwave_extractor, lidar_extractor, csi_extractor):
        super(feature_extrator, self).__init__()
        self.rgb_extractor = rgb_extractor
        self.depth_extractor = depth_extractor
        self.mmwave_extractor = mmwave_extractor
        self.lidar_extractor = lidar_extractor
        self.csi_extractor = csi_extractor
    def forward(self, rgb_data, depth_data, mmwave_data, lidar_data, csi_data):
        rgb_feature = self.rgb_extractor(rgb_data)
        depth_feature = self.depth_extractor(depth_data)
        mmwave_feature = self.mmwave_extractor(mmwave_data)
        lidar_feature = self.lidar_extractor(lidar_data)
        csi_feature = self.csi_extractor(csi_data)
        return rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature

### linear projection on seq len dim

In [ ]:
class linear_projection(nn.Module):
    def __init__(self, rgb_dim, depth_dim, mmwave_dim, lidar_dim, csi_dim, output_dim):
        super(linear_projection, self).__init__()
        self.output_dim = output_dim
        self.rgb_linear = nn.Linear(rgb_dim, output_dim)
        self.depth_linear = nn.Linear(depth_dim, output_dim)
        self.mmwave_linear = nn.Linear(mmwave_dim, output_dim)
        self.lidar_linear = nn.Linear(lidar_dim, output_dim)
        self.csi_linear = nn.Linear(csi_dim, output_dim)
    def forward(self, rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature, modality_list):
        rgb_feature = self.rgb_linear(rgb_feature.permute(0, 2, 1))
        depth_feature = self.depth_linear(depth_feature.permute(0, 2, 1))
        mmwave_feature = self.mmwave_linear(mmwave_feature.permute(0, 2, 1))
        lidar_feature = self.lidar_linear(lidar_feature.permute(0, 2, 1))
        csi_feature = self.csi_linear(csi_feature.permute(0, 2, 1))
        features = torch.cat([rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature], dim=2)
        empty_feature = torch.zeros_like(rgb_feature, device=torch.device('cuda'))
        for i in range(len(modality_list)):
            if modality_list[i] == True:
                continue
            else:
                features[:, :, i*self.output_dim:(i+1)*self.output_dim] = empty_feature
        features = features.permute(0, 2, 1)
        return features

### linear projection on patch embedding dim

In [14]:
class linear_projection(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(linear_projection, self).__init__()
        self.linear_projection = nn.Linear(input_dim, output_dim)
    def forward(self, rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature, modality_list):
        feature_list = [rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature]
        if len (modality_list) == 0:
            print('WARNING: modality_list is empty!')
            feature = torch.zeros_like(lidar_feature, device=torch.device('cuda'))
            feature = self.linear_projection(feature)
        else:
            real_feature_list = []
            for i in range(len(modality_list)):
                if modality_list[i] == True:
                    real_feature_list.append(feature_list[i])
                else:
                    continue
            feature = torch.cat(real_feature_list, dim=1)
            feature = self.linear_projection(feature)
        return feature

### test

In [17]:
rgb_data = torch.rand(16, 3, 480, 640).cuda()
depth_data = torch.rand(16, 3, 480, 640).cuda()
mmwave_data = torch.rand(16, 63, 5).cuda()
lidar_data = torch.rand(16, 1467, 3).cuda()
csi_data = torch.rand(16, 3, 114, 10).cuda()

feature_extractor = feature_extrator(rgb_extractor, depth_extractor, mmwave_extractor, lidar_extractor, csi_extractor).cuda()
rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature = feature_extractor(rgb_data, depth_data, mmwave_data, lidar_data, csi_data)

modality_list = [True, True, True, True, True]

linear_projector = linear_projection(512, 128).cuda()
feature = linear_projector(rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature, modality_list)
print(feature.shape)

torch.Size([16, 261, 128])


c:\Users\Chen_Xinyan\anaconda3\envs\pytorch\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


### ViT components

In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size = 225, num_heads = 4, dropout = 0.0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.qkv = nn.Linear(emb_size, emb_size*3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)
    
    def forward(self, x, mask = None):
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)
        
        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
        
    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion = 4, drop_p = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )
        
class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size = 225,
                 drop_p = 0.5,
                 forward_expansion = 4,
                 forward_drop_p = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))
        
class TransformerEncoder(nn.Sequential):
    def __init__(self, depth = 1, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])
        
class regression_Head(nn.Sequential):
    def __init__(self, emb_size, num_classes):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size), 
            nn.Linear(emb_size, num_classes))
        
class ViT(nn.Sequential):
    def __init__(self,
                emb_size = 128,
                depth = 1,
                *,
                num_classes = 17*3,
                **kwargs):
        super().__init__(
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            regression_Head(emb_size, num_classes)
        )

In [21]:
vit = ViT().cuda()
out = vit(feature)
print(out.shape)

torch.Size([16, 51])


In [22]:
class modality_invariant_model(nn.Module):
    def __init__(self, feature_extractor, linear_projector, vit):
        super(modality_invariant_model, self).__init__()
        self.feature_extractor = feature_extrator(rgb_extractor, depth_extractor, mmwave_extractor, lidar_extractor, csi_extractor)
        self.linear_projector = linear_projector
        self.vit = vit
    def forward(self, rgb_data, depth_data, mmwave_data, lidar_data, csi_data, modality_list):
        rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature = self.feature_extractor(rgb_data, depth_data, mmwave_data, lidar_data, csi_data)
        feature = self.linear_projector(rgb_feature, depth_feature, mmwave_feature, lidar_feature, csi_feature, modality_list)
        out = self.vit(feature)
        return out

In [25]:
rgb_data = torch.rand(16, 3, 480, 640).cuda()
depth_data = torch.rand(16, 3, 480, 640).cuda()
mmwave_data = torch.rand(16, 63, 5).cuda()
lidar_data = torch.rand(16, 1467, 3).cuda()
csi_data = torch.rand(16, 3, 114, 10).cuda()
modality_list = random.choices(
    [True, False],
    k= 5,
    weights=[80, 20]
)
model = modality_invariant_model(feature_extractor, linear_projector, vit).cuda()
out = model(rgb_data, depth_data, mmwave_data, lidar_data, csi_data, modality_list)
print(out.shape)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.43 GiB. GPU 0 has a total capacty of 23.99 GiB of which 0 bytes is free. Of the allocated memory 21.83 GiB is allocated by PyTorch, and 360.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [21]:
import random
result = random.choices(
    [True, False],
    k= 5,
    weights=[80, 20]
)
print(result)

[True, False, True, True, True]


#### check parameters （prove to be the same)

In [139]:
for param in csi_model.parameters():
    print(param)
    print(a)

Parameter containing:
tensor([[[[ 0.1442, -0.0400,  0.0629],
          [-0.1392,  0.0949,  0.2094],
          [ 0.0652, -0.2481, -0.3184]]],


        [[[ 0.0702, -0.2592, -0.1461],
          [ 0.0290, -0.2670, -0.1601],
          [-0.0984, -0.1065,  0.1317]]],


        [[[-0.2569, -0.1761, -0.2460],
          [-0.0025,  0.0818, -0.1916],
          [ 0.0658,  0.0344, -0.1465]]],


        [[[ 0.1116, -0.2394, -0.2183],
          [-0.0552, -0.1350, -0.1518],
          [ 0.0485, -0.0301, -0.2722]]],


        [[[ 0.1532, -0.1014, -0.0618],
          [ 0.3229, -0.0714,  0.1710],
          [-0.1570,  0.2686,  0.3306]]],


        [[[ 0.1332, -0.3275, -0.0421],
          [-0.2583, -0.3117, -0.2423],
          [-0.1054,  0.2280,  0.0535]]],


        [[[-0.1630, -0.1043,  0.2208],
          [ 0.1549, -0.2130,  0.0888],
          [-0.1122, -0.2864,  0.3296]]],


        [[[ 0.2309, -0.1619, -0.2679],
          [-0.2516,  0.0649,  0.2708],
          [ 0.0679, -0.0229, -0.0502]]],


        [[

NameError: name 'a' is not defined

In [140]:
for param in csi_extractor.parameters():
    print(param)
    print(a)

Parameter containing:
tensor([[[[ 0.1442, -0.0400,  0.0629],
          [-0.1392,  0.0949,  0.2094],
          [ 0.0652, -0.2481, -0.3184]]],


        [[[ 0.0702, -0.2592, -0.1461],
          [ 0.0290, -0.2670, -0.1601],
          [-0.0984, -0.1065,  0.1317]]],


        [[[-0.2569, -0.1761, -0.2460],
          [-0.0025,  0.0818, -0.1916],
          [ 0.0658,  0.0344, -0.1465]]],


        [[[ 0.1116, -0.2394, -0.2183],
          [-0.0552, -0.1350, -0.1518],
          [ 0.0485, -0.0301, -0.2722]]],


        [[[ 0.1532, -0.1014, -0.0618],
          [ 0.3229, -0.0714,  0.1710],
          [-0.1570,  0.2686,  0.3306]]],


        [[[ 0.1332, -0.3275, -0.0421],
          [-0.2583, -0.3117, -0.2423],
          [-0.1054,  0.2280,  0.0535]]],


        [[[-0.1630, -0.1043,  0.2208],
          [ 0.1549, -0.2130,  0.0888],
          [-0.1122, -0.2864,  0.3296]]],


        [[[ 0.2309, -0.1619, -0.2679],
          [-0.2516,  0.0649,  0.2708],
          [ 0.0679, -0.0229, -0.0502]]],


        [[

NameError: name 'a' is not defined